In [ ]:
import pandas as pd

In [ ]:
question_answer = pd.read_csv('csv/Posts.csv')

In [ ]:
question_answer.set_index('Id', inplace=True)

In [ ]:
answers = question_answer[question_answer['PostTypeId'] == 2]
questions = question_answer[question_answer['PostTypeId'] == 1]
# this observation has come from the fact that, PostTypeId == 2, have a parent Id, while those with 1 have answerCount field non empty, rest all ids are wikis etc.

In [ ]:
answerer = answers.groupby('OwnerUserId')['ParentId'].apply(list).to_dict()
answerer

In [ ]:
question_tags = questions['Tags'].apply(lambda x: list(filter(lambda x: x != '', x.split('|'))))
question_tags

In [ ]:
exploded_tags = question_tags.explode()
exploded_tags

In [ ]:
tags = exploded_tags.value_counts()
tags

In [ ]:
count_tag = tags.to_dict()
count_tag

In [ ]:
answers.isna().sum()
# 6822 answers are such that their owner is missing

In [ ]:
count_answers_per_user = answers['OwnerUserId'].value_counts().to_dict()

In [ ]:
count_answers_per_user

In [ ]:
#! wrong code, gives very wrong answers, design is the most used tag

# count_tag = dict()
# for tags in question_tags:
#   for tag in tags:
#     if tag not in count_tag:
#       count_tag[tag] = 0
#     count_tag[tag] += 1

In [ ]:
count_tag

In [ ]:
tags_df = pd.read_csv('csv/Tags.csv')

In [ ]:
# tags_df.set_index('Id', inplace=True)
tag_cols = tags_df.columns
tag_cols

In [ ]:
req_tag_cols = ['Id', 'TagName']
drop_cols = list(filter(lambda x: x not in req_tag_cols, tag_cols))
drop_cols


In [ ]:
new_tags_df = tags_df.drop(drop_cols, axis=1)
new_tags_df

In [ ]:
new_tags_df.set_index('TagName', inplace=True)
new_tags_df

In [ ]:
tag_dict = new_tags_df.to_dict()['Id']
tag_dict

In [ ]:
count_tag = {tag_dict[k]: v for k, v in count_tag.items() if k in tag_dict} # some tags not there in the tags.csv file, don't know why :(
count_tag

In [ ]:
answerer_table = pd.DataFrame(count_answers_per_user.items(), columns=['UserId', 'AnsweredQuestionCount'])
answerer_table

In [ ]:
tags_table = pd.DataFrame(count_tag.items(), columns=['TagId', 'Count'])
tags_table

In [ ]:
sorted_tags_table = tags_table.sort_values(by='Count', ascending=False)
sorted_tags_table.head(3)

In [ ]:
sorted_answerer_table = answerer_table.sort_values(by='AnsweredQuestionCount', ascending=False)
sorted_answerer_table.head(3)

# Second Part

In [ ]:
threshold = 20

In [ ]:
answerer_table = answerer_table[answerer_table['AnsweredQuestionCount'] >= threshold]
answerer_table

In [ ]:
tags_table = tags_table[tags_table['Count'] >= threshold]
tags_table

In [ ]:
print(tag_dict)

In [ ]:
user_tag_count = {}
tags_table = tags_table.sort_values(['TagId'])
answerer_table = answerer_table.sort_values(['UserId'])
users = answerer_table['UserId'].values
tags = tags_table['TagId'].values
q_tag_map = question_tags.to_dict()


for user in users:
  tag_count = {tag : 0 for tag in tags}
  for question in answerer[user]:
    for tag in q_tag_map[question]:
      try:
        tag_count[tag_dict[tag]] += 1
      except KeyError:
        pass
  user_tag_count[user] = tag_count
del q_tag_map
del users
del tags

In [ ]:
user_tag_count

In [ ]:
expert = pd.DataFrame.from_dict(user_tag_count, orient='index')
del user_tag_count

In [ ]:
expert

In [ ]:
expert.shape # 1163 users, 973 tags

# Third Part

In [ ]:
expert = expert.map(lambda x: x/3 if x < 15 else 5)
expert

In [ ]:
expert_matrix = expert.to_numpy()
expert_shape = expert_matrix.shape
expert

In [ ]:
test_start = (int(0.85 *expert_shape[0] ), int(0.85*expert_shape[1]))
test_users, test_tags = test_start
test_start

In [ ]:
test_matrix = expert_matrix[test_users:, test_tags:]
test_matrix.shape

In [ ]:
"""
Report the following about your utility matrix:
Summation value of the utility matrix
Highest row sum of the utility matrix
Highest column sum of the utility matrix

"""

sum_utility = expert_matrix.sum()
highest_row_sum = expert_matrix.sum(axis=1).max()
highest_col_sum = expert_matrix.sum(axis=0).max()
sum_utility, highest_row_sum, highest_col_sum

In [ ]:
"""
Report the following for your train and test data
Summation value of the train matrix
Dimension of the test matrix
Summation value of test matrix
"""

sum_train = expert_matrix.sum() - test_matrix.sum()
dim_test = test_matrix.shape
sum_test = test_matrix.sum()
sum_train, dim_test, sum_test


# Fourth Part

In [126]:
from abc import ABC, abstractmethod
import numpy as np
class CollaborativeFilter():
  def __init__(self, utility_matrix):
    utility_zero_mask = utility_matrix == 0
    utility_matrix[utility_zero_mask] = np.nan
    self.utility_matrix = utility_matrix
    self.utility_matrix_filled = utility_matrix.copy()

  @abstractmethod
  def fit(self):
    pass

  @abstractmethod
  def predict(self):
    pass
  
  #TODO: Complete the compute_similarities method, find the cosine similarity between the user_vector and all other users in the utility_matrix
  def compute_similarities(self, user_vector : np.ndarray):
    pass

class UserBasedCollaborativeFilter(CollaborativeFilter):
    def __init__(self,expert_matrix ,k ):
      """
      utility_matrix : 2D numpy array
      k : int
      
      k is the number of similar users to consider for prediction
      utility_matrix is the matrix of user ratings, nan filled with 0
      """
      super().__init__(expert_matrix)
      self.k = k
      
    #TODO: Complete the predict method, find the k most similar users and predict the rating for the item(also handle the case when less than k users are available)
    def predict(self, user_vector : np.ndarray, i: int):
      """
      user_vector : 1D numpy array
      i : int
      
      i is the index of the item to predict the rating for
      """
      self.similarities = self.compute_similarities(user_vector)
      similar_users = np.argsort(self.similarities)
      users_to_consider = self.utility_matrix[i, :] != np.nan
      
      similarity_scores = self.similarities[similar_users]
      
      

,name,phone
0,1.0,1
1,2.0,2
2,3.0,3
3,4.0,4
4,NaN,5


In [140]:
test = [
     [4, np.nan, 5, 1, np.nan, np.nan, np.nan],
     [5, 5, 4, np.nan, np.nan, np.nan, np.nan],
]

arr = np.array(test)
np.corrcoef(arr)

array([[nan, nan],
       [nan, nan]])

nan

In [129]:
df.corr()

,name,phone
name,1.0,1.0
phone,1.0,1.0
